<a href="https://colab.research.google.com/github/yal521/DSCI441-machine-learning-project/blob/main/swarm_feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Feature Selection

In [7]:
pip install niapy --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
#from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
import pandas as pd

from niapy.problems import Problem
from niapy.task import Task
from niapy.algorithms.basic import ParticleSwarmOptimization


class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [9]:
from random import sample
Credit_data = pd.read_csv("/content/drive/MyDrive/Yang_DSCI441_Final_Project/dataset1_creditcard.csv")


In [10]:
Credit_data.shape

(284807, 31)

In [29]:
Credit_data1=Credit_data.sample(n=int(284807*0.1))
Transaction = Credit_data1.drop(['Class'],axis ='columns')
Label = Credit_data1 ['Class']
feature_names = Transaction.columns

In [30]:
Credit_data1.shape

(28480, 31)

In [31]:
a = Transaction.to_numpy()
b = Label.to_numpy()
c = feature_names.to_numpy()


a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=0.2, stratify=b, random_state=1234)

problem = SVMFeatureSelection(a_train,b_train)
task = Task(problem, max_iters=100)
algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
best_features, best_fitness = algorithm.run(task)

selected_features = best_features > 0.5
print('Number of selected features:', selected_features.sum())
print('Selected features:', ', '.join(c[selected_features].tolist()))

model_selected = SVC()
model_all = SVC()

model_selected.fit(a_train[:, selected_features], b_train)
print('Subset accuracy:', model_selected.score(a_test[:, selected_features], b_test))

model_all.fit(a_train, b_train)
print('All Features Accuracy:', model_all.score(a_test, b_test))

Number of selected features: 4
Selected features: V4, V5, V16, V18
Subset accuracy: 0.9985955056179775
All Features Accuracy: 0.9982443820224719
